## Score en kaggle: 0.85481

In [2]:
import catboost as cb
import xgboost as xgb
import pandas as pd

In [17]:
import scipy.stats as stats
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [6]:
labels = pd.read_csv('labels_training_set.csv')

In [7]:
processed = pd.read_csv('features_04.csv')

In [9]:
processed = processed.set_index(processed.columns[0])
processed = processed.drop(['primer_visita','ultima_visita'],axis=1)
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.0,...,1362,False,False,True,False,False,True,False,True,False
00091926,viewed product,iPhone 6S,Computer,0,34,372,448,28,0,16.0,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,10,1,66,10.0,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,206,10,5,20.6,...,1155,False,True,True,True,True,True,True,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.0,...,12924,True,True,True,True,False,True,False,True,False


# XGBoost

In [10]:
training_features = labels.set_index('person').join(processed,how='left')
predict_features = kaggle.set_index('person').join(processed,how='left')

In [11]:
training_labels = training_features['label']
training_features = training_features.drop(['label'],axis=1)

## Mean encoding

In [14]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [15]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.049121,0.056883,0.036301,1,17,23,68,9,0,7.555556,...,865,False,True,True,True,True,True,True,True,False
1,0.041894,0.042654,0.036301,0,1,0,2,1,3,2.000000,...,12924,False,True,False,False,False,False,False,True,False
2,0.049121,0.045238,0.036301,0,22,31,96,109,2,0.880734,...,12924,True,True,True,True,False,True,True,True,False
3,0.049121,0.090909,0.036301,0,4,24,37,9,4,4.111111,...,12924,False,False,False,True,False,True,True,True,False
4,0.049121,0.060423,0.068258,0,1,9,17,1,12,17.000000,...,59,False,True,True,False,False,False,False,True,False


## Prediccion (pre-entrenado)

In [21]:
predictor = xgb.XGBClassifier()

In [22]:
predictor.load_model('preentrenados/85193xgboost')

In [24]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [25]:
predict_features['label']=prediction_submit

In [29]:
submit_xgb = kaggle.set_index('person')

In [30]:
submit_xgb['label'] = predict_features['label']

# CatBoost

In [31]:
catBoost = cb.CatBoostClassifier()
catBoost.load_model('preentrenados/85317catboost')

In [32]:
pred = catBoost.predict_proba(predict_features)[:,1]
submit_cb = kaggle.set_index('person')
submit_cb['label'] = pred

In [34]:
submit_cb.head()

,label
person,
4886f805,0.009441
0297fc1e,0.071510
2d681dd8,0.016723
cccea85e,0.064428
4c8a8b93,0.030369


In [35]:
submit_xgb.head()

,label
person,
4886f805,0.007937
0297fc1e,0.053218
2d681dd8,0.015391
cccea85e,0.106399
4c8a8b93,0.023377


In [46]:
submit = 0.6*submit_cb+0.4*submit_xgb

In [47]:
submit.to_csv('submit.csv')